In [ ]:
#@markdown #**Perfil Aerodinàmic**
#@markdown ---
#@markdown **RECORDA:**  Cal clicar el botó "Play" o "Run all" per que l'applet funcioni.

#@markdown Aquest applet mostra el perfil aerodinàmic de l'ala d'un avió (o d'un ocell!). \
#@markdown Les fletxes mostren el camp de velocitat, i el mapa de colors el camp de pressió. \
#@markdown Com pots veure, la pressió a la part superior de l'ala és menor que a la part de sota. \
#@markdown Això fa que aparegui una força cap amunt anomenada "sustentació", que permet el vol.

#@markdown **Prova-ho tu!**

#@markdown Prova a modificar l'angle d'atac de l'ala i mira com canvia el camp de pressió. \
#@markdown Com creus que afectarà això a la força de sustentació sobre l'ala?

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Parameters of the grid
shape        = (400, 200)
xrange       = (-8, 8)
yrange       = (-3, 5)

# Parameters of the airfoil
mu    = -0.15 + 0.15j
cut_h = 0.07
V_inf = 3
R         = np.sqrt((1 - mu.real)**2 + mu.imag**2)
theta     = np.linspace(0, 2*np.pi, 100)
zeta_circ = mu + R * np.exp(1j * theta)
z_shape   = (zeta_circ + 1/zeta_circ)

# Define the complex plane grid
rez  = np.linspace(*xrange, shape[0])[:, None]
imz  = np.linspace(*yrange, shape[1])[None, :]
z    = (rez + 1j * imz)

# Inverse Joukowsky transform
def z2zeta(z):
    discriminant = np.sqrt(z**2 - 4)
    zeta = (z + discriminant) / 2
    in_unit_circle = (np.abs(zeta) <= 1)
    below_wing = (z.real**2 < 4) & (z.imag > 0) & \
              (z.imag < cut_h * (4 - z.real**2))
    bad_branch = below_wing ^ in_unit_circle
    zeta[bad_branch] -= discriminant[bad_branch]
    return zeta

# W = vx - i vy
def complex_velocity(z, alpha):
    zeta     = z2zeta(z)
    gamma = 4 * np.pi * V_inf * R * np.sin( alpha + np.arcsin(mu.imag / R) )
    W_tilde  = V_inf * np.exp(-1j * alpha)
    W_tilde += 1j * gamma / (2 * np.pi * (zeta - mu))
    W_tilde -= V_inf * R**2 * np.exp(1j * alpha) / (zeta - mu)**2
    W = W_tilde / (1 - 1/zeta**2)
    return W.conj()

# Bernoullis's principle
def pressure(z, alpha):
    st = complex_velocity(z, alpha)
    P = 0.5 * (V_inf**2 - np.abs(st)**2)
    return P

##### Do the plotting #####
fig1, ax1 = plt.subplots(figsize = (10,5))
plt.rc('font', size = 12)
plt.close()

def plot_wing(alpha_deg = 20):

    alpha = alpha_deg * (np.pi / 180)
    rot = np.exp(1j * alpha)
    z_airfoil = z_shape * rot.conj()
    W = complex_velocity(z * rot, alpha) / rot

    ax1.cla()
    fig1.set_size_inches(10, 5)
    ax1.set_xlim(*xrange)
    ax1.set_ylim(*yrange)
    ax1.imshow(pressure(z * rot, alpha).T, origin = 'lower', cmap = 'jet', extent = xrange + yrange, vmin = -10, vmax = 7)
    ax1.streamplot(z.real.T, z.imag.T, W.real.T, W.imag.T, color = 'k', linewidth = 0.7)
    ax1.fill(z_airfoil.real, z_airfoil.imag, color = 'white', zorder=2)

    return fig1

interact(
    plot_wing,
    alpha_deg = FloatSlider(value=20, min=0, max=45, step=1,
        description="Angle d'atac (en graus)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);

interactive(children=(FloatSlider(value=20.0, description="Angle d'atac (en graus)", layout=Layout(width='800p…

In [ ]:
#@markdown #**Turbulència**
#@markdown ---
#@markdown **RECORDA:**  Cal clicar el botó "Play" o "Run all" per que l'applet funcioni.

#@markdown Aquest applet mostra un fluid que es desplaça cap a la dreta i troba un obstacle en el camí. \
#@markdown Això fa que aparegui una filera de remolins, anomenada tren de vòrtexs de Von Kármán. \
#@markdown És un dels casos més típics on es genera turbulència quan la viscositat és prou baixa.

#@markdown **Prova-ho tu!**

#@markdown Observa el moviment del fluid a diferents instants de temps, i la formació dels vòrtexs.

import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

shape = (200, 50)
iters = 3000
rho0  = 100
tau   = 0.55
frame_every = 30

x     = np.arange(shape[0])[:, None]
y     = np.arange(shape[1])[None, :]
curl  = np.zeros(shape + (iters//frame_every,))

D2Q9  = np.array([[di, dj, 4 / (9 * 4**(di**2 + dj**2))] for di in (-1,0,1) for dj in (-1,0,1)])
stone = (x - 25)**2 + (y - 25)**2 < 5**2

# Initial conditions
F = np.ones(shape + (9,)) + 0.01 * np.random.randn(*shape, 9)
F[:,:,7] += 2 * (1 + 0.2 * np.cos(2 * np.pi * x / shape[0] * 4))
F *= rho0 / np.sum(F, 2)[..., None]

print('Evolucionant el fluid...')
for it in tqdm(range(iters)):

    # Advection
    for k, (di, dj, w) in enumerate(D2Q9):
        F[:,:,k] = np.roll(F[:,:,k], int(di), axis = 0)
        F[:,:,k] = np.roll(F[:,:,k], int(dj), axis = 1)

    # Save stone
    stoneF = F[stone,:]

    # Hydro variables
    rho = np.sum(F, 2)
    ux  = np.sum(F * D2Q9[:, 0], 2) / rho
    uy  = np.sum(F * D2Q9[:, 1], 2) / rho

    # Collisions
    Feq = rho[..., None] * D2Q9[:, 2] * (1 + 3 * (D2Q9[:, 0] * ux[..., None] + D2Q9[:, 1] * uy[..., None])    +  \
                                         (9/2) * (D2Q9[:, 0] * ux[..., None] + D2Q9[:, 1] * uy[..., None])**2 -  \
                                         (3/2) * (ux[..., None]**2 + uy[..., None]**2))
    F += - (1 / tau) * (F - Feq)

    # Apply boundaries
    F[stone,:] = stoneF[:, ::-1]

    #Compute vorticity
    if it % frame_every == 0:
        dvx_dy = np.roll(ux, -1, axis = 1) - np.roll(ux, 1, axis = 1)
        dvy_dx = np.roll(uy, -1, axis = 0) - np.roll(uy, 1, axis = 0)
        curl[..., it//frame_every] =  dvx_dy - dvy_dx

##### Do the plotting #####
fig2, ax2 = plt.subplots(figsize = (20,5))
plt.rc('font', size = 12)
plt.close()

stoneshape = 25 + 25j + 6 * np.exp(1j * np.linspace(0, 2*np.pi, 100))
ax2.fill(stoneshape.real, stoneshape.imag, color = 'wheat', zorder = 2)
ish2 = ax2.imshow(curl[..., 0].T, cmap='ocean', interpolation = 'bicubic', origin = 'lower', vmin = -.08, vmax = .08)

def plot_flow(frame = 0):
    fig2.set_size_inches(20, 5)
    ish2.set_data(curl[..., int(frame)].T)
    return fig2

interact(
    plot_flow,
    frame = FloatSlider(value = 70, min = 0, max = iters//frame_every - 1, step=1,
        description="Temps (unitats arbitràries)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);


Evolucionant el fluid...


  0%|          | 0/3000 [00:00<?, ?it/s]

interactive(children=(FloatSlider(value=70.0, description='Temps (unitats arbitràries)', layout=Layout(width='…

In [17]:
#@markdown #**Espectre de Planck**
#@markdown ---
#@markdown **RECORDA:**  Cal clicar el botó "Play" o "Run all" per que l'applet funcioni.

#@markdown Aquest applet mostra la distribució de longituds d'ona que emet un objecte a temperatura T. \
#@markdown La funció que descriu aquesta distribució s'anomena espectre de Planck, o llei de Planck. \
#@markdown La deducció d'aquesta llei per Max Planck, l'any 1900, va conduir al descobriment de la física quàntica.

#@markdown **Prova-ho tu!**

#@markdown Modifica la temperatura i mira com canvia la distribució d'emissió. \
#@markdown Fixa't en la posició del pic sobre l'espectre visible. Correspon al que esperaries?

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

lam = np.linspace(10, 2000, 100)
cmap = mpl.colormaps['jet']
wien = 2.897771955e6

def Planck(lam, T):
    h, c, k = 6.62607015e-34, 2.99792458e8, 1.380649e-23
    lam_m   = lam * 1e-9
    result  = (2 * h * c**2 / lam_m**5) / (np.exp(h * c / (lam_m * k * T)) - 1)
    return result / 1e13

##### Do the plotting #####
fig3, ax3 = plt.subplots(figsize = (8, 6))
plt.rc('font', size = 15)
plt.close()

for lam in np.linspace(400, 750, 70):
    ax3.plot(2 * [lam], [0.0, 3.5], color = cmap(int( 255 * (lam - 400) / 350 )), alpha = 0.7, zorder = 1)

lam = np.linspace(10, 2000, 100)
ln31, = ax3.plot(lam, Planck(lam, 5000), 'k')
ln32, = ax3.plot([wien/5000, wien/5000], [0.0, 3.5], 'k--')

ax3.set_xlim(0, 2000)
ax3.set_ylim(0, 3.5)
ax3.set_yticks([])
ax3.set_facecolor('wheat')
ax3.set_xlabel("Longitud d'ona (nm)")
ax3.set_ylabel("Intensitat")

def plot_planck(T):
    fig3.set_size_inches(10, 5)
    ln31.set_data(lam, Planck(lam, T))
    ln32.set_data([wien/T, wien/T], [0.0, 3.5])
    return fig3

interact(
    plot_planck,
    T   = FloatSlider(value = 5000, min = 3000, max = 7000, step=100,
        description="Temperatura (Kelvin)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);


interactive(children=(FloatSlider(value=5000.0, description='Temperatura (Kelvin)', layout=Layout(width='800px…

In [16]:
#@markdown #**Refracció en una lent**
#@markdown ---
#@markdown **RECORDA:**  Cal clicar el botó "Play" o "Run all" per que l'applet funcioni.

#@markdown Aquest applet mostra una simulació de la llum en passar a través d'una lent convergent. \
#@markdown L'index de refracció del vidre és d'aproximadament n = 1,5, cosa que fa desviar els raigs de llum. \
#@markdown Podem aprofitar aquesta propietat per construir lents que concentrin la llum en el seu punt focal.

#@markdown **Prova-ho tu!**

#@markdown Observa l'evolució de les ones al llarg del temps, i com interactuen amb la lent. \
#@markdown Les línies grogues marquen la trajectòria de dos raigs de llum. Quina relació tenen amb els fronts d'ona?


import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

shape        = (300, 300)
xrange       = (-10, 10)
yrange       = (-10, 10)

CFL          = 0.5
tmax         = 20
save_every   = 2

###### Prepare the grids
x = np.linspace(*xrange, shape[0] + 1)[:-1, None]
y = np.linspace(*yrange, shape[1] + 1)[None, :-1]

dx, dy = x[1, 0] - x[0, 0], y[0, 1] - y[0, 0]
dt     = CFL * np.minimum(dx, dy)
iters  = np.int64(tmax // dt) + 1
frames = (iters - 1) // save_every + 1

F         = np.zeros(shape + (2,), dtype=np.float32)
stored    = np.zeros(shape + (frames,))

###### Prepare the lens
speed   = 1
ref_ind = 1.5
R       = 11
lens    = ((x - 5)**2 + y**2 < R**2) * ((x + 15)**2 + y**2 < R**2)
f       = R / (2 * (ref_ind - 1))

###### Wave Equations
def rhs(t, F):
    u, v    = F.transpose(2,0,1)
    d2u_dx2 = (np.roll(u, 1, axis=0) - 2*u + np.roll(u, -1, axis=0)) / dx**2
    d2u_dy2 = (np.roll(u, 1, axis=1) - 2*u + np.roll(u, -1, axis=1)) / dy**2
    lap_u   = d2u_dx2 + d2u_dy2

    dX_dt         = np.zeros_like(F)
    dX_dt[..., 0] = v
    dX_dt[..., 1] = speed**2 / (1 + (ref_ind - 1) * lens)**2 * lap_u

    return dX_dt

###### Runge-Kutta 4th order step
def RK4(t, F):
    k1 = rhs(t, F)
    k2 = rhs(t + 0.5 * dt, F + 0.5 * k1 * dt)
    k3 = rhs(t + 0.5 * dt, F + 0.5 * k2 * dt)
    k4 = rhs(t + dt, F + k3 * dt)
    return t + dt, F + dt * (k1 + 2*k2 + 2*k3 + k4) / 6

###### Boundary Conditions
def apply_boundary(t, F):
    k = 2 * np.pi
    w = k * speed
    F[:2, :, 0]  =     + np.sin( k * x[:2] - w * t)
    F[:2, :, 1]  = - w * np.cos( k * xrange[0] - w * t)
    F[-2:, :, 0] = 0
    F[-2:, :, 1] = 0

    return t, F

###### Time evolution
print('Evolucionant la llum...')
t = 0.0
for i in tqdm(range(iters)):
    t, F = RK4(t, F)
    t, F = apply_boundary(t, F)
    if i % save_every == 0:
        stored[..., i//save_every] = F[..., 0]

##### Do the plotting #####

fig4, ax4 = plt.subplots(1, 1, figsize= (6,6))
plt.rc('font', size = 15)
plt.close()

ish4 = ax4.imshow(stored[..., 0].T, cmap = 'summer', interpolation = 'bilinear' , origin = 'lower', extent = xrange + yrange, vmin = -1, vmax = 1)
ax4.imshow(np.where(lens.T, 1, np.nan), cmap = 'cool', alpha = 0.5, interpolation = 'bilinear', origin = 'lower', extent = xrange + yrange)
ax4.plot([-10, -5, f - 5, -5, -10], [2.7, 2.7, 0, -2.7, -2.7], color = 'yellow')

def plot_lens(frame):
    fig4.set_size_inches(6, 6)
    ish4.set_data(stored[..., int(frame)].T)
    return fig4

interact(
    plot_lens,
    frame = FloatSlider(value = frames//2, min = 0, max = frames - 1 , step=1,
        description="Temps (u. a.)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);

Evolucionant la llum...


  0%|          | 0/601 [00:00<?, ?it/s]

interactive(children=(FloatSlider(value=150.0, description='Temps (u. a.)', layout=Layout(width='800px'), max=…

In [15]:
#@markdown #**Efecte Doppler**
#@markdown ---
#@markdown **RECORDA:**  Cal clicar el botó "Play" o "Run all" per que l'applet funcioni.

#@markdown L'efecte Doppler és l'aparent canvi de ferqüència sonora que percebem quan l'emissor o el receptor es mouen. \
#@markdown És el responsable de la característica baixada de to que notem quan passa un vehicle a gran velocitat. \
#@markdown Aquest applet mostra els fronts d'ona que es generen en un avió en moviment.

#@markdown **Prova-ho tu!**

#@markdown Modifica la velocitat de l'avió i observa què els passa als fronts d'ona davant i darrere seu. \
#@markdown Com creus que sentirem el so dels motors abans que l'avió passi? I després? \
#@markdown Prova ara d'augmentar la velocitat de l'avió per sobre de la velocitat del so. Què observes?


#@title Efecte Doppler
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

theta = np.linspace(0, 2 * np.pi, 100)

##### Do the plotting #####
fig5, ax5 = plt.subplots(figsize = (12, 6.2))
plt.rc('font', size = 15)
plt.close()

def plot_doppler(vel = 0.5):

    ax5.cla()
    fig5.set_size_inches(12, 6.2)
    ax5.set_facecolor('cyan')
    ax5.set_xlim(-3.0, 1.1)
    ax5.set_ylim(-1.1, 1.1)
    ax5.fill([-0.25, -0.25, 0.1], [0.02, -0.02, 0], color = 'k')
    ax5.fill([-0.15, -0.15, -0.10], [0.2, -0.2, 0], color = 'k')
    ax5.fill([-0.25, -0.25, -0.2], [0.05, -0.05, 0], color = 'k')
    fronts = [ax5.plot( r * np.cos(theta) - vel * r, r * np.sin(theta) , 'b', lw = 0.5) for r in np.arange(0, 2, 0.1)]
    if vel > 1:
        mach = ax5.plot([-3, 0, -3], [3 * np.tan(np.arcsin(1/vel)), 0, -3 * np.tan(np.arcsin(1/vel))], '--k')

    ax5.arrow(0, 0, vel/2, 0, color = 'red', width = .01, zorder=3)
    ax5.text(0.3, 0.1, f'v = {vel:.2f} v$_s$', fontdict = {'size': 12, 'color': 'red'})

    return fig5

interact(
    plot_doppler,
    vel = FloatSlider(value=0.8, min=0, max=2, step=0.1,
        description="Velocitat (mach)",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);

interactive(children=(FloatSlider(value=0.8, description='Velocitat (mach)', layout=Layout(width='800px'), max…